In [1]:
import numpy as np
from SALib.sample import morris
from SALib.analyze import morris as morris_analyze

from scipy.interpolate import LinearNDInterpolator
from vtk import vtkXMLUnstructuredGridReader

import scipy.stats as stats
import subprocess
import scipy as sp
import rasterio
import meshio
import pandas as pd
from matplotlib import pyplot as plt
import pyvista

import os
import shutil
import datetime
import math


from SALib.sample import morris as morris_sampler
from SALib.analyze import morris as morris_analyzer

import pyvista as pv

    

## define the model

In [2]:
# Define your model or function that takes an input array and returns an output

meshC =  pyvista.read(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\gridTetraCoarse.vtu")

# In[21]:



dfRiver = pd.read_csv('C:\\Users\\Andrea\\Documents\\Flow\\Optimization\\headData\\averageInTime\\wellHeadRivers2013_2020.csv')
wellLocRiver = dfRiver
wellLocRiver['Bottom'] = wellLocRiver['quota[m]'] - wellLocRiver['depth']

df = pd.read_csv('C:\\Users\\Andrea\\Documents\\Flow\\Optimization\\headData\\averageInTime\\wellHeadPo2013_2020.csv')
wellLoc = df
wellLoc['Bottom'] = wellLoc['quota[m]'] - wellLoc['depth']


def model(P,*args):

    myfile = open(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po1HP.prj", "rt")
    contents = myfile.readlines()
    myfile.close()
    weights = np.loadtxt(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\weights.txt")
    nRecostruction = sum(weights[:,1])

    wordX = 'permeability' ## it is set according to the uncertainty probability.
    nameOfRivers = ['Adige',
                    'Secchia',
                    'DoraBaltea',
                    'Ticino',
                    'Chiese',
                    'Oglio',
                    'Tanaro',
                    'Po1',
                    'Po2',
                    'Po3',
                    'Reno',
                    'Lamone',
                    'Savio',
                    'Adda',
                    'Taro',
                    'Mincio',
                    'Sesia',
                    'Orco',
                    'Lambro',
                    'NaviglioGrande',
                    'none']

    al = 0
    worda = 'alpha' + nameOfRivers[al]
    wordqm = 'qms'
    wordh0 = 'h0'
    j=0
    qms = 0
    h0 = 0
    P0 = h0*9810
    
    # set the K parameters 
    Kappa1 = (10**P[0]*weights[0,:] + 10**P[1]*weights[1,:] + 
              10**P[2]*weights[2,:] + 10**P[3]*weights[3,:] + 
              10**P[4]*weights[4,:] + 10**P[5]*weights[5,:])/nRecostruction
    meshC['Kappa1'] = Kappa1
    meshC.save(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\gridTetraCoarse.vtu")
    for i in range(len(contents)):
        
#         if wordX in contents[i]:
#             value = contents[i+2].split('<value>').pop().split('</value>')[0]
#             if j>0:
#                 K = 10**P[j-1]
#                 value2= str(K)
#                 contents[i+2] = contents[i+2].replace(value, value2)
#                 permName = 'k%1d' % j 
#                 print(permName, value2)
#             j += 1

        #set the alpha values
        if worda in contents[i]:
            if '<value>' in contents[i+2]:
                value = contents[i+2].split('<value>').pop().split('</value>')[0]
                a = 10**args[0][al]
                
                #a = args[al]

                value2= str(a)
                contents[i+2] = contents[i+2].replace(value, value2)
                #print(worda, value2)
                al += 1
                worda = 'alpha' + nameOfRivers[al]
               
        #set the qms values          
        if wordqm in contents[i]:
              if '<value>' in contents[i+2]:
                    value = contents[i+2].split('<value>').pop().split('</value>')[0]
                    qmsValue2 = P[-1]*float(value)
                    value2= str(qmsValue2)
                    contents[i+2] = contents[i+2].replace(value, value2)                    
                    #print('C_qms', qms+1 ,':', P[-1],';', 'qms', qms+1 , ':' , value2)
                    qms += 1
                    
        if wordh0 in contents[i]:
              if '<value>' in contents[i+2]:
                    value = contents[i+2].split('<value>').pop().split('</value>')[0]
                    value2= str(P0)
                    contents[i+2] = contents[i+2].replace(value, value2)                  
    
    
            
    with open(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po2.prj", 'w') as file:
        file.writelines( contents )
    
    
    subprocess.check_call([r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs-6.4.3-16-g9a7564950-Windows-10.0.22000-python-3.10.7-utils\bin\ogs.exe", r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po2.prj"])

    sample = np.array([wellLoc.longitude,wellLoc.latitude,wellLoc.Bottom]).T
    reader = vtkXMLUnstructuredGridReader()
    reader.SetFileName(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po_ts_1_t_1.000000.vtu")
    reader.Update()

    # Get the point coordinates and pressure data
    points = reader.GetOutput().GetPoints().GetData()
    pressure = reader.GetOutput().GetPointData().GetArray("pressure")

    # Create an interpolator object
    interp = LinearNDInterpolator(points, pressure)

    # Sample the interpolated value in multiple 3D points of your grid
    points = sample
    interpolated_values = interp(points)
    simulatedP = interpolated_values
    #simulatedP = (simulatedP>=0)*simulatedP+(simulatedP<0)*0
    simulatedH = (simulatedP)/9810+sample[:,2]
    #simulatedH = (simulatedH<=wellLoc['quota[m]'])*simulatedH+(simulatedH>wellLoc['quota[m]'])*wellLoc['quota[m]']
    sum_of_non_nan_elements = np.nansum(((simulatedH-np.array(wellLoc['averageHead[m]']))**2)**(1/2))
    number_of_non_nan_elements = np.count_nonzero(~np.isnan(simulatedH))
    fobj = sum_of_non_nan_elements / number_of_non_nan_elements
    
    
    # Read the VTU file
    meshP = pv.read(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po_ts_1_t_1.000000.vtu")
    

    print('MAE: ',fobj)
    return meshP['pressure']


test the model

In [3]:

P = [1.468076415412256e-09,
        5.63755330166905e-09,
        3.5010182167685024e-13,
       2.1387780771147673e-10, 
        8.12861733198715e-10,
        6.818104881251327e-12,
    1.4999359889106798]

P = [1.0175798951590066e-09,
     3.834766652828529e-09,
     2.2403786421078086e-13,
     1.5685931737322937e-10,
     2.125822903488283e-15,
     5.044748775747056e-18,
     0.9998005094218905]


args1 = [-3.42857048, -3.99471462, -9.06199728, -7.86643814, -5.7780723,  -7.95268559,
 -5.31287564, -4.78660503, -4.88061929, -6.58881351, -6.22568987, -5.1196719,
 -4.41932612, -5.00985718, -5.83508412, -9.87809193, -3.04361441, -4.63172516,
 -8.93327461, -6.99463648]  

args1 = [-6.08714914, -6.14730799, -5.98892462, -8.96016767, -4.47571414, -8.61162733,
 -6.516952,   -3.52932064, -3.871093,   -5.93522541, -6.53802047, -6.25750149,
 -3.12460615, -6.49405604, -6.73598515, -9.13666717, -3.66514438, -5.74559475,
 -5.3577788,  -7.98832236,  4.96167782]

P1 = np.log10(P)
P1[-1] = P[-1]


p = model(P1,args1)

print(p)

MAE:  6.633082580611168
[ 266228.69798536  203699.6459359   188637.99531098 ...   26997.3770716
 -489939.65013289   76224.68559305]


In [4]:
for i in range(len (P)):
    print(np.log10(P[i])+0.2)
    print(np.log10(P[i])-0.2)
    print()

-8.792431482186776
-9.192431482186775

-8.216261057913751
-8.61626105791375

-12.449678576183148
-12.849678576183146

-9.604489679302244
-10.004489679302242

-14.47247291819531
-14.872472918195308

-17.097160456389094
-17.497160456389093

0.19991335369988772
-0.2000866463001123



In [5]:
10**-17.497160456389093

3.1830212911150737e-18

In [6]:
# import numpy as np
# from SALib.sample import morris as morris_sampler
# from SALib.analyze import morris as morris_analyzer



# # Define the number of samples and levels for the Morris method
# num_samples = 1000
# num_levels = 4

# # Define the ranges for each input parameter

# # Define the ranges for each parameter
# param_ranges = [[-8.792431482186776, -9.192431482186775],
#                          [-8.216261057913751, -8.61626105791375],
#                          [-12.449678576183148,-12.849678576183146],
#                          [-9.604489679302244,-10.004489679302242],
#                          [-14.47247291819531,-14.872472918195308],
#                          [-17.097160456389094,-17.497160456389093],
#                          [1.2, 0.8]
#                         ]

# # Generate the Morris samples using the specified parameter ranges
# samples = morris_sampler.sample(num_samples, num_levels, param_ranges)

# # Evaluate the model for each sample
# model_outputs = np.zeros(num_samples)
# for i, sample in enumerate(samples):
#     model_outputs[i] = model(sample,args1)

# # Perform the Morris analysis
# morris_results = morris_analyzer.analyze(
#     parameter_ranges, samples, model_outputs, num_levels=num_levels, conf_level=0.95
# )

# # Print the results
# for i, param in enumerate(morris_results["names"]):
#     print(f"Parameter {i+1} ({param}):")
#     print(f"  Mu: {morris_results['mu'][i]}")
#     print(f"  Sigma: {morris_results['sigma'][i]}")
#     print()


In [24]:
# # Define the number of samples and levels for the Morris method
# num_samples = 1000
# num_levels = 4

# # Define the parameter names and ranges
# problem = {
#     'num_vars': 7,
#     'names': ['k 1', 'k 2', 'k 3', 'k 4', 'k 5', 'k 6', 'c'],
#     'bounds': [[-9.203212302198214,-8.203212302198214],
#                          [-8.557603941922235,-7.557603941922235],
#                          [-12.960415758067137,-11.960415758067137],
#                          [-9.880580941153951,-8.880580941153951],
#                          [-10.681610259869824,-9.681610259869824],
#                          [-17.480981875487103,-16.480981875487103],
#                          [ 1.491223510483059,2.491223510483059]
#                         ]
# }

# # Generate the Morris samples using the specified parameter ranges
# samples = morris_sampler.sample(problem, num_samples, num_levels=num_levels)


# # Evaluate the model for each sample
# model_outputs = np.zeros([num_samples, len(p)])
# for i, sample in enumerate(samples):
#     model_outputs[i] = model(sample,args1)


# # Perform the Morris analysis
# morris_results = morris_analyzer.analyze(problem, samples, model_outputs, num_levels=num_levels, conf_level=0.95)

# # Print the results
# for i, param in enumerate(morris_results["names"]):
#     print(f"Parameter {i+1} ({param}):")
#     print(f"  Mu: {morris_results['mu'][i]}")
#     print(f"  Sigma: {morris_results['sigma'][i]}")
#     print()


In [8]:
import numpy as np
from SALib.sample import morris as morris_sampler
from SALib.analyze import morris

# Define your model evaluation function
evaluate_model = model

# Define the problem definition
problem = {
    'num_vars': 7,  # Number of input parameters
    'names': ['param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'param7'],  # Parameter names
    'bounds': [[-9.192431482186775,-8.792431482186776],
                         [-8.61626105791375,-8.216261057913751],
                         [-12.849678576183146,-12.449678576183148],
                         [-10.004489679302242,-9.604489679302244],
                         [-14.872472918195308,-14.47247291819531],
                         [-17.497160456389093,-17.097160456389094],
                         [0.8,1.2]]
,  # Parameter bounds
    'groups': None  # Parameter grouping (optional)
}

# Define the number of samples and levels for the Morris method
n_samples = 500
n_levels = 4

# Generate the Morris samples using the provided sampler
param_values = morris_sampler.sample(problem, n_samples, n_levels)
print('trajectory calculated succesfully!')
# Evaluate the model for each sample
model_outputs = np.array([evaluate_model(x,args1) for x in param_values])
print('Done compute morris simulations!')


trajectory calculated succesfully!
MAE:  9.907156941267148
MAE:  10.480496930600246
MAE:  7.169300634365893
MAE:  7.169173019273198
MAE:  7.16927373383309
MAE:  7.018938949239357
MAE:  7.01895242995512
MAE:  8.019637740978288
MAE:  6.874119626321399
MAE:  6.8741198895660744
MAE:  6.874119515170039
MAE:  8.846910450463321
MAE:  8.5998076523735
MAE:  8.599810243018664
MAE:  11.085179429180544
MAE:  9.912755191406367
MAE:  6.932191212508549
MAE:  6.932185599411071
MAE:  7.294474885489865
MAE:  7.1673101847634015
MAE:  7.167312992605639
MAE:  8.396836568992914
MAE:  8.396859635786738
MAE:  7.466371450582109
MAE:  7.719287370217881
MAE:  7.719220396663791
MAE:  7.239736950084788
MAE:  6.7497087943236505
MAE:  6.749705047482233
MAE:  6.464055055859679
MAE:  8.146582426377778
MAE:  8.146586409164378
MAE:  9.343342470552018
MAE:  9.800337125925367
MAE:  9.80034991743772
MAE:  6.882095777024334
MAE:  7.724116733777761
MAE:  7.724463013872435
MAE:  7.723967752557196
MAE:  7.535830483980345
MAE: 

MAE:  6.852882159063737
MAE:  7.608599907444901
MAE:  7.774097489135349
MAE:  9.800337238569767
MAE:  9.80033554604586
MAE:  11.608970732126252
MAE:  7.724440500080077
MAE:  7.723951704050895
MAE:  7.724457406461508
MAE:  7.265900679388317
MAE:  7.244779631117244
MAE:  7.359385271944994
MAE:  7.750019602243642
MAE:  7.827322891309949
MAE:  7.827313944085865
MAE:  7.827312023971454
MAE:  9.004995404030092
MAE:  7.239723152537707
MAE:  7.239081049078002
MAE:  12.990071352463188
MAE:  8.088782613405517
MAE:  8.082834469830035
MAE:  8.08878782652487
MAE:  7.2087127078828175
MAE:  6.87854251726653
MAE:  6.746247011509551
MAE:  6.746248912208045
MAE:  7.961488549025813
MAE:  8.739749518539924
MAE:  8.739748586636702
MAE:  6.932188120285275
MAE:  7.294466627377658
MAE:  7.294483823487695
MAE:  7.97101958441298
MAE:  7.971026989407517
MAE:  10.528907892357916
MAE:  7.019354735209186
MAE:  7.609272430011317
MAE:  7.075470396293898
MAE:  7.075467483306672
MAE:  7.075466100108891
MAE:  6.74702173

MAE:  6.883076069128881
MAE:  7.630079658897311
MAE:  7.960275092594104
MAE:  7.960465654186925
MAE:  7.960147621940146
MAE:  10.49307648598034
MAE:  8.260366676769461
MAE:  8.26037415404794
MAE:  8.204265774036166
MAE:  8.204266144649425
MAE:  8.20426628197473
MAE:  8.015233245496606
MAE:  7.741629693908447
MAE:  8.130470430840527
MAE:  6.995484587831736
MAE:  6.9960653408102464
MAE:  6.995488597102033
MAE:  7.075983302011909
MAE:  6.8528705679345965
MAE:  7.608732176141093
MAE:  7.608808344985974
MAE:  7.604232032026676
MAE:  7.9092458179253935
MAE:  7.909247497922179
MAE:  7.9092362056210845
MAE:  7.909233865294784
MAE:  7.39713275819654
MAE:  8.019531893525697
MAE:  7.018975762008275
MAE:  8.128702166044649
MAE:  8.128730390139204
MAE:  8.128409268601727
MAE:  7.026192879397609
MAE:  7.026335133556294
MAE:  7.1005740105662865
MAE:  8.603523201687947
MAE:  8.993790035405494
MAE:  6.534087919488651
MAE:  7.151305081207016
MAE:  7.257064713022305
MAE:  7.257438259439073
MAE:  6.874125

MAE:  9.131822315555041
MAE:  9.13202205592192
MAE:  9.132006307456228
MAE:  7.7988947531441495
MAE:  7.798881690302723
MAE:  7.629381969080199
MAE:  8.550034978807366
MAE:  7.03444247373826
MAE:  8.800646218124868
MAE:  8.80041080242196
MAE:  8.80065310583512
MAE:  8.14673703769552
MAE:  7.359231374006719
MAE:  7.359379910040613
MAE:  7.750031337856084
MAE:  8.229552505816393
MAE:  7.518256065544809
MAE:  7.518253081536755
MAE:  7.518252176823812
MAE:  7.518233685315852
MAE:  7.975294910248554
MAE:  7.496389358155793
MAE:  7.275701837382591
MAE:  6.5692304903957055
MAE:  8.229548158230644
MAE:  8.229517066057968
MAE:  8.140079782088216
MAE:  8.140076827518707
MAE:  8.140077661365726
MAE:  8.809349502334944
MAE:  7.49096370135115
MAE:  8.8092751426141
MAE:  8.809351514965611
MAE:  8.80932249009125
MAE:  8.140081139661028
MAE:  8.229554769690235
MAE:  6.569248153153955
MAE:  7.975310258637873
MAE:  7.975298636812125
MAE:  7.614302840897663
MAE:  7.880698096067984
MAE:  8.269519607808954

MAE:  10.724007803245533
MAE:  6.84771184907452
MAE:  6.439102327635867
MAE:  8.140082642431716
MAE:  7.273785507439259
MAE:  7.273837847220388
MAE:  7.273837082777857
MAE:  7.4909507192901845
MAE:  7.637805906585967
MAE:  7.63814896069232
MAE:  7.512092690734517
MAE:  9.34327339899786
MAE:  6.50468084430936
MAE:  6.5045492372981695
MAE:  6.504184575308999
MAE:  6.504552353248843
MAE:  6.882094639369912
MAE:  7.724467899755452
MAE:  6.776349155184865
MAE:  10.235864806140668
MAE:  8.442172184472689
MAE:  8.442175873419648
MAE:  8.442179577934672
MAE:  8.846920083483658
MAE:  8.846920124018753
MAE:  8.599810243018664
MAE:  6.895945903909073
MAE:  7.961506263851056
MAE:  8.388690741899024
MAE:  8.389017009381554
MAE:  8.38900618341075
MAE:  9.363241400834507
MAE:  9.363204284212355
MAE:  7.673765061064791
MAE:  8.491985707963169
MAE:  8.49198456064408
MAE:  9.562671016084131
MAE:  7.209620154512118
MAE:  7.208801220739487
MAE:  8.088781374039156
MAE:  7.644626619402541
MAE:  7.6461012214

...

MAE:  7.169273302626068
MAE:  7.397134208695569
MAE:  7.397152204289143
MAE:  7.397149058987717
MAE:  7.909255254957724
MAE:  7.604236556790345
MAE:  7.668539509341262
MAE:  7.668551424882568
MAE:  7.485086919779826
MAE:  7.485082561376351
MAE:  7.964273041521771
MAE:  7.96427417430449
MAE:  7.964277795399018
MAE:  7.20667871300185
MAE:  7.156316054678368
MAE:  7.445800259740453
MAE:  7.556854614425749
MAE:  7.301781163005741
MAE:  7.301781152514774
MAE:  7.301779700641999
MAE:  7.301785122755182
MAE:  7.213491641126833
MAE:  7.614310590769774
MAE:  8.149767920419176
MAE:  6.429977584396184
MAE:  6.652632074229746
MAE:  6.652273345591305
MAE:  7.382847537885314
MAE:  7.383645263536419
MAE:  7.618313899159783
MAE:  7.618308856039149
MAE:  8.163889541867894
MAE:  12.058729566826623
MAE:  12.058724317691542
MAE:  10.528890601716512
MAE:  8.63340421611547
MAE:  8.31720618423448
MAE:  8.317220297511797
MAE:  8.317241864833184
MAE:  7.164821394577644
MAE:  7.608600416767157
MAE:  6.852710460

MAE:  9.912765992092712
MAE:  10.235876202852275
MAE:  7.755042547490631
MAE:  9.76582213986165
MAE:  9.765821398158659
MAE:  7.440262293752394
MAE:  6.747483798456914
MAE:  7.075509330600433
MAE:  7.075486803780194
MAE:  7.075485172936089
MAE:  8.993796818129766
MAE:  8.99404332531697
MAE:  8.994043808973746
MAE:  8.603525353542171
MAE:  6.852882324598148
MAE:  7.608607684278886
MAE:  8.12836837090554
MAE:  8.128409517457609
MAE:  9.188648843971855
MAE:  9.563047887050795
MAE:  9.562682631717054
MAE:  9.562682262860196
MAE:  12.991140773515017
MAE:  8.083418055887456
MAE:  8.083401121987542
MAE:  6.848669699207051
MAE:  7.896793872154733
MAE:  7.909233846957786
MAE:  7.909230729637157
MAE:  10.48047121906156
MAE:  10.480494731129243
MAE:  9.907156922165218
MAE:  9.907230580990047
MAE:  6.754432315249197
MAE:  11.639791428279784
MAE:  11.085104681419663
MAE:  11.085181328223262
MAE:  6.9983265019939065
MAE:  6.997925196668493
MAE:  6.533480141749095
MAE:  6.533832983764766
MAE:  7.1516

MAE:  8.142411144752195
MAE:  8.142394795373209
MAE:  7.650497888990926
MAE:  7.65108040071305
MAE:  7.63775627729548
MAE:  8.867077679915516
MAE:  7.099442583773703
MAE:  7.098631695610675
MAE:  7.099440780078503
MAE:  7.099440791622057
MAE:  6.569245528735415
MAE:  6.43910284423191
MAE:  8.280082452768253
MAE:  9.18864984598355
MAE:  9.188647857333649
MAE:  9.188654709018119
MAE:  6.9833382297249225
MAE:  6.982930186957054
MAE:  7.6461069894053875
MAE:  8.088793790815094
MAE:  6.88307554502235
MAE:  6.883074884591446
MAE:  6.8830848900454775
MAE:  6.88307777508236
MAE:  8.070602669435042
MAE:  8.375537082244174
MAE:  10.079919424385432
MAE:  7.594955299194628
MAE:  7.644625796530183
MAE:  6.982885000077567
MAE:  6.98333137136376
MAE:  6.982924562012342
MAE:  6.746246964469675
MAE:  8.28008873480121
MAE:  8.491986403539984
MAE:  8.491979168394062
MAE:  8.388653707587487
MAE:  7.961496541476086
MAE:  8.739745847065413
MAE:  8.739715105115526
MAE:  6.932181193613368
MAE:  6.932182658989

MAE:  7.1692887063626465
MAE:  6.75443469091653
MAE:  6.75442514662796
MAE:  6.695957143166687
MAE:  6.695951319646448
MAE:  6.695954763407313
MAE:  7.829548579462339
MAE:  7.019329256772264
MAE:  7.42515276452425
MAE:  7.425045252149067
MAE:  8.632206711705903
MAE:  9.08982413516575
MAE:  9.089819188625638
MAE:  9.089825116460432
MAE:  8.6903293968696
MAE:  11.33100766736747
MAE:  11.33096575907152
MAE:  8.690329684386906
MAE:  8.690331935000732
MAE:  8.690320416607118
MAE:  8.317201401076652
MAE:  8.633415880089645
MAE:  7.425109842234872
MAE:  7.534043286836846
MAE:  7.309919869288186
MAE:  7.309521704439885
MAE:  7.310088961381101
MAE:  6.504676613562692
MAE:  9.343274557464
MAE:  9.343355828983908
MAE:  10.904620948885077
MAE:  10.724017255732782
MAE:  10.346478050354646
MAE:  12.333470821420917
MAE:  7.645833841756473
MAE:  7.646116884420429
MAE:  7.646084158980108
MAE:  6.9833328113039235
MAE:  6.982924701194503
MAE:  8.994043808973746
MAE:  7.075994453732195
MAE:  6.99581821794

MAE:  7.100096340298096
MAE:  7.4663884264877085
MAE:  7.778547969608765
MAE:  7.630080694565906
MAE:  7.630079371578489
MAE:  9.525105604132056
MAE:  9.52509964269029
MAE:  9.524203137523019
MAE:  8.070594086600028
MAE:  8.37554504847822
MAE:  9.764189021439606
MAE:  9.764186411069252
MAE:  6.65530081639777
MAE:  6.6553015359921535
MAE:  7.1191277397938
MAE:  7.848797759525852
MAE:  7.848473841502836
MAE:  7.887929214893094
MAE:  7.11912965171721
MAE:  7.119130596171464
MAE:  7.225193692252172
MAE:  7.887520436500056
MAE:  7.8879525446324505
MAE:  7.749689921786507
MAE:  7.749945820395297
MAE:  9.258730140903157
MAE:  6.819944003357385
MAE:  6.921888423457067
MAE:  6.921445849910965
MAE:  7.2251943012996
MAE:  7.225190270750308
MAE:  7.119128739889041
MAE:  7.119128367896948
MAE:  7.543951218415908
MAE:  7.31566341550022
MAE:  7.3156610246069524
MAE:  7.315656479009563
MAE:  11.639791428279784
MAE:  11.639791774521267
MAE:  8.846914504181644
MAE:  8.441968591826425
MAE:  8.27235599036

In [9]:
param_values

array([[ -9.19243148,  -8.48292772, -12.84967858, ..., -14.47247292,
        -17.49716046,   1.06666667],
       [ -9.19243148,  -8.48292772, -12.84967858, ..., -14.47247292,
        -17.49716046,   1.06666667],
       [ -8.92576482,  -8.48292772, -12.84967858, ..., -14.47247292,
        -17.49716046,   1.06666667],
       ...,
       [ -9.19243148,  -8.21626106, -12.84967858, ..., -14.73913958,
        -17.49716046,   0.93333333],
       [ -9.19243148,  -8.21626106, -12.84967858, ..., -14.73913958,
        -17.49716046,   1.2       ],
       [ -8.92576482,  -8.21626106, -12.84967858, ..., -14.73913958,
        -17.49716046,   1.2       ]])

In [10]:
from tqdm import tqdm
model_outputs_meters = model_outputs/9810

# Create an empty array to store the results
mu_star = np.zeros([problem['num_vars'],np.shape(model_outputs)[1]])
sigma = np.zeros([problem['num_vars'],np.shape(model_outputs)[1]])

# Iterate over each output column
for i in tqdm(range(np.shape(model_outputs_meters)[1]), desc="Progress", unit="output"):
    # Perform Morris analysis for each output column
    morris_indices = morris.analyze(problem, param_values, model_outputs_meters[:,i],
                                    conf_level=0.95, print_to_console=False)
    
    # Store the results in the respective arrays
    mu_star[:,i] = morris_indices['mu_star']
    sigma[:,i] = morris_indices['sigma']





Progress: 100%|█████████████████████████████████████████████████████████████| 36204/36204 [05:20<00:00, 112.85output/s]


In [11]:
meshP = pv.read(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po_ts_1_t_1.000000.vtu")
meshMu = meshP
for par in range(problem['num_vars']):
    colNameMu = f'mu_star_P{par+1}'
    colNameS = f'sigma_P{par+1}'
    meshMu[colNameMu] = mu_star[par,:]
    meshMu[colNameS] = sigma[par,:]
    
output_path = r'D:\Andrea\Flow\SensitivityAnalisys\meshSA.vtu'
meshMu.save(output_path)

In [ ]:
import pandas as pd

def save_matrix_to_csv(matrix, filename):
    df = pd.DataFrame(matrix)
    df.to_csv(filename, index=False)

# Example usage
filename = r'D:\Andrea\Flow\SensitivityAnalisys\morris.csv'
save_matrix_to_csv(model_outputs, filename)
print(f"Matrix saved to '{filename}' successfully.")

## evaluate the SA for the rivers 

In [6]:
# Define your model or function that takes an input array and returns an output

meshC =  pyvista.read(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\gridTetraCoarse.vtu")

# In[21]:



dfRiver = pd.read_csv('C:\\Users\\Andrea\\Documents\\Flow\\Optimization\\headData\\averageInTime\\wellHeadRivers2013_2020.csv')
wellLocRiver = dfRiver
wellLocRiver['Bottom'] = wellLocRiver['quota[m]'] - wellLocRiver['depth']

df = pd.read_csv('C:\\Users\\Andrea\\Documents\\Flow\\Optimization\\headData\\averageInTime\\wellHeadPo2013_2020.csv')
wellLoc = df
wellLoc['Bottom'] = wellLoc['quota[m]'] - wellLoc['depth']


def model1(P):
    
    args = [ -8.99243148,  -8.41626106, -12.64967858,  -9.80448968, -14.67247292, -17.29716046,   0.99980051]

    myfile = open(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po1HP.prj", "rt")
    contents = myfile.readlines()
    myfile.close()
    weights = np.loadtxt(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\weights.txt")
    nRecostruction = sum(weights[:,1])
    
    

    wordX = 'permeability' ## it is set according to the uncertainty probability.
    nameOfRivers = ['Adige',
                    'Secchia',
                    'DoraBaltea',
                    'Ticino',
                    'Chiese',
                    'Oglio',
                    'Tanaro',
                    'Po1',
                    'Po2',
                    'Po3',
                    'Reno',
                    'Lamone',
                    'Savio',
                    'Adda',
                    'Taro',
                    'Mincio',
                    'Sesia',
                    'Orco',
                    'Lambro',
                    'NaviglioGrande',
                    'none']

    al = 0
    worda = 'alpha' + nameOfRivers[al]
    wordqm = 'qms'
    wordh0 = 'h0'
    j=0
    qms = 0
    h0 = 0
    P0 = h0*9810
    #args = args2[0]
    
    
    
    # set the K parameters 
    Kappa1 = (10**args[0]*weights[0,:] + 10**args[1]*weights[1,:] + 
              10**args[2]*weights[2,:] + 10**args[3]*weights[3,:] + 
              10**args[4]*weights[4,:] + 10**args[5]*weights[5,:])/nRecostruction
    meshC['Kappa1'] = Kappa1
    meshC.save(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\gridTetraCoarse.vtu")
    for i in range(len(contents)):
        
#         if wordX in contents[i]:
#             value = contents[i+2].split('<value>').pop().split('</value>')[0]
#             if j>0:
#                 K = 10**P[j-1]
#                 value2= str(K)
#                 contents[i+2] = contents[i+2].replace(value, value2)
#                 permName = 'k%1d' % j 
#                 print(permName, value2)
#             j += 1

        #set the alpha values
        if worda in contents[i]:
            if '<value>' in contents[i+2]:
                value = contents[i+2].split('<value>').pop().split('</value>')[0]
                a = 10**P[al]
                
                #a = args[al]

                value2= str(a)
                contents[i+2] = contents[i+2].replace(value, value2)
                #print(worda, value2)
                al += 1
                worda = 'alpha' + nameOfRivers[al]
               
        #set the qms values          
        if wordqm in contents[i]:
              if '<value>' in contents[i+2]:
                    value = contents[i+2].split('<value>').pop().split('</value>')[0]
                    qmsValue2 = args[-1]*float(value)
                    value2= str(qmsValue2)
                    contents[i+2] = contents[i+2].replace(value, value2)                    
                    #print('C_qms', qms+1 ,':', args[-1],';', 'qms', qms+1 , ':' , value2)
                    qms += 1
                    
        if wordh0 in contents[i]:
              if '<value>' in contents[i+2]:
                    value = contents[i+2].split('<value>').pop().split('</value>')[0]
                    value2= str(P0)
                    contents[i+2] = contents[i+2].replace(value, value2)                  
    
    
            
    with open(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po2.prj", 'w') as file:
        file.writelines( contents )
    
    
    subprocess.check_call([r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs-6.4.3-16-g9a7564950-Windows-10.0.22000-python-3.10.7-utils\bin\ogs.exe", r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po2.prj"])

    sample = np.array([wellLoc.longitude,wellLoc.latitude,wellLoc.Bottom]).T
    reader = vtkXMLUnstructuredGridReader()
    reader.SetFileName(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po_ts_1_t_1.000000.vtu")
    reader.Update()

    # Get the point coordinates and pressure data
    points = reader.GetOutput().GetPoints().GetData()
    pressure = reader.GetOutput().GetPointData().GetArray("pressure")

    # Create an interpolator object
    interp = LinearNDInterpolator(points, pressure)

    # Sample the interpolated value in multiple 3D points of your grid
    points = sample
    interpolated_values = interp(points)
    simulatedP = interpolated_values
    #simulatedP = (simulatedP>=0)*simulatedP+(simulatedP<0)*0
    simulatedH = (simulatedP)/9810+sample[:,2]
    #simulatedH = (simulatedH<=wellLoc['quota[m]'])*simulatedH+(simulatedH>wellLoc['quota[m]'])*wellLoc['quota[m]']
    sum_of_non_nan_elements = np.nansum(((simulatedH-np.array(wellLoc['averageHead[m]']))**2)**(1/2))
    number_of_non_nan_elements = np.count_nonzero(~np.isnan(simulatedH))
    fobj = sum_of_non_nan_elements / number_of_non_nan_elements
    
    
    # Read the VTU file
    meshP = pv.read(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po_ts_1_t_1.000000.vtu")
    

    print('MAE: ',fobj)
    return meshP['pressure']


In [7]:


args1 = [1.0175798951590066e-09,
     3.834766652828529e-09,
     2.2403786421078086e-13,
     1.5685931737322937e-10,
     2.125822903488283e-15,
     5.044748775747056e-18,
     0.9998005094218905]


 

P = [-6.08714914, -6.14730799, -5.98892462, -8.96016767, -4.47571414, -8.61162733,
 -6.516952,   -3.52932064, -3.871093,   -5.93522541, -6.53802047, -6.25750149,
 -3.12460615, -6.49405604, -6.73598515, -9.13666717, -3.66514438, -5.74559475,
 -5.3577788,  -7.98832236]

args2 = np.log10(args1)
args2[-1] = args1[-1]

P = [-6.08714909, -6.14730804, -5.98892457, -8.96016772, -4.47571419, -8.61162735,
 -6.51695205, -3.52932059, -3.87109302, -5.93522543, -6.53802045, -6.25750151,
 -3.12460613, -6.49405602, -6.73598517, -9.13666712, -3.66514436, -5.7455947,
 -5.35777882, -7.98832238]


p = model1(P)

print(p)

MAE:  6.633090847667728
[ 266228.57866173  203699.82961062  188637.90834279 ...   26997.11114166
 -489940.09533873   76225.23184243]


In [8]:
for i in range(len(P)):
    print([P[i]-2,P[i]+2],',')

[-8.08714909, -4.08714909] ,
[-8.14730804, -4.14730804] ,
[-7.98892457, -3.98892457] ,
[-10.96016772, -6.960167719999999] ,
[-6.47571419, -2.4757141899999997] ,
[-10.61162735, -6.611627349999999] ,
[-8.51695205, -4.51695205] ,
[-5.52932059, -1.5293205900000002] ,
[-5.87109302, -1.87109302] ,
[-7.93522543, -3.93522543] ,
[-8.538020450000001, -4.53802045] ,
[-8.25750151, -4.25750151] ,
[-5.12460613, -1.12460613] ,
[-8.49405602, -4.49405602] ,
[-8.73598517, -4.73598517] ,
[-11.13666712, -7.13666712] ,
[-5.66514436, -1.6651443600000002] ,
[-7.7455947, -3.7455947] ,
[-7.35777882, -3.35777882] ,
[-9.98832238, -5.98832238] ,


In [10]:
import numpy as np
from SALib.sample import morris as morris_sampler
from SALib.analyze import morris

# Define your model evaluation function
evaluate_model = model1

# Define the problem definition
problem = {
    'num_vars': 20,  # Number of input parameters
    'names': ['Adige',
                    'Secchia',
                    'DoraBaltea',
                    'Ticino',
                    'Chiese',
                    'Oglio',
                    'Tanaro',
                    'Po1',
                    'Po2',
                    'Po3',
                    'Reno',
                    'Lamone',
                    'Savio',
                    'Adda',
                    'Taro',
                    'Mincio',
                    'Sesia',
                    'Orco',
                    'Lambro',
                    'NaviglioGrande'],  # Parameter names
    'bounds': [[-8.08714909, -4.08714909] ,
                [-8.14730804, -4.14730804] ,
                [-7.98892457, -3.98892457] ,
                [-10.96016772, -6.960167719999999] ,
                [-6.47571419, -2.4757141899999997] ,
                [-10.61162735, -6.611627349999999] ,
                [-8.51695205, -4.51695205] ,
                [-5.52932059, -1.5293205900000002] ,
                [-5.87109302, -1.87109302] ,
                [-7.93522543, -3.93522543] ,
                [-8.538020450000001, -4.53802045] ,
                [-8.25750151, -4.25750151] ,
                [-5.12460613, -1.12460613] ,
                [-8.49405602, -4.49405602] ,
                [-8.73598517, -4.73598517] ,
                [-11.13666712, -7.13666712] ,
                [-5.66514436, -1.6651443600000002] ,
                [-7.7455947, -3.7455947] ,
                [-7.35777882, -3.35777882] ,
                [-9.98832238, -5.98832238] ]
                    ,  # Parameter bounds
    'groups': None  # Parameter grouping (optional)
}

# Define the number of samples and levels for the Morris method
n_samples = 500
n_levels = 4

# Generate the Morris samples using the provided sampler
param_values = morris_sampler.sample(problem, n_samples, n_levels)
print('trajectory calculated succesfully!')
# Evaluate the model for each sample
model_outputs = np.array([evaluate_model(x) for x in param_values])
print('Done compute morris simulations!')


trajectory calculated succesfully!
MAE:  6.889529782594267
MAE:  6.885997225789349
MAE:  7.454711126721952
MAE:  7.44152686606811
MAE:  7.115698050093981
MAE:  7.168048698598787
MAE:  7.177478579359979
MAE:  7.176927784922697
MAE:  7.177307295478947
MAE:  7.195401847088344
MAE:  7.159707827478331
MAE:  7.365074896437069
MAE:  7.368917333231782
MAE:  7.384271879200778
MAE:  7.384151548596972
MAE:  7.381270638922915
MAE:  7.317029611154577
MAE:  7.313315285033271
MAE:  7.284203767476013
MAE:  7.309227077332078
MAE:  7.320614990247581
MAE:  6.7266849443631
MAE:  7.062948585123764
MAE:  6.8535939559782255
MAE:  6.848559985056737
MAE:  6.931636441062682
MAE:  6.911187645226418
MAE:  6.860709929593342
MAE:  6.878611243705755
MAE:  6.882722860250832
MAE:  6.8824472526842175
MAE:  6.882632239080949
MAE:  6.876209793852235
MAE:  6.865816624637649
MAE:  7.090301197714023
MAE:  7.091417861111013
MAE:  7.090644574967818
MAE:  7.094034323882254
MAE:  7.094278628480973
MAE:  7.173124407728564
MAE:  

MAE:  6.654242108209245
MAE:  6.992519556405365
MAE:  7.00379201193622
MAE:  6.9394118198247705
MAE:  6.963391234378931
MAE:  6.963402993311182
MAE:  6.9074485177621385
MAE:  6.8777477169786625
MAE:  6.862996103090108
MAE:  6.862797433743526
MAE:  6.861939553111212
MAE:  6.86228377200425
MAE:  7.07194113150354
MAE:  7.087933703354726
MAE:  7.087788099321065
MAE:  7.097693594215449
MAE:  7.290763391068334
MAE:  6.978780774164958
MAE:  6.978726446840875
MAE:  6.983089357245736
MAE:  6.9814803000321355
MAE:  6.995272629018879
MAE:  6.997000600333245
MAE:  6.996790044668224
MAE:  6.853082424246943
MAE:  6.858771398139417
MAE:  6.860043223496222
MAE:  6.834947039345381
MAE:  6.834447863551317
MAE:  6.752921734616365
MAE:  6.727217549120001
MAE:  6.7108782348272875
MAE:  6.7089184982490675
MAE:  6.709125129389462
MAE:  6.740805674531583
MAE:  6.787632271222083
MAE:  6.775913643612028
MAE:  7.330327822027651
MAE:  7.33268989220086
MAE:  7.6213112696157275
MAE:  7.283109116550003
MAE:  7.64665

MAE:  7.701264405619608
MAE:  7.068252392410874
MAE:  6.863259626502246
MAE:  6.870153465899725
MAE:  6.904751623072178
MAE:  6.904738861576182
MAE:  6.8135211176415575
MAE:  6.7892158116997345
MAE:  6.777810062507742
MAE:  6.761983661153867
MAE:  7.088064012879711
MAE:  7.395954159356996
MAE:  7.402631944630938
MAE:  7.258441193215514
MAE:  7.2585463216069925
MAE:  7.196306453579736
MAE:  7.188798705507835
MAE:  7.21488034291927
MAE:  7.20214797725742
MAE:  7.295314753752565
MAE:  7.293351722626578
MAE:  7.142691178053449
MAE:  7.14325513087072
MAE:  7.1493830025510645
MAE:  7.192722545309497
MAE:  7.238212081655463
MAE:  7.233519553365063
MAE:  7.244004031527525
MAE:  7.244063267831192
MAE:  6.959928475039835
MAE:  6.974074884303778
MAE:  7.292472284652102
MAE:  7.290358929381647
MAE:  7.1303422043452915
MAE:  7.131641372735552
MAE:  7.143654256999532
MAE:  7.145249069710868
MAE:  7.146971566245741
MAE:  7.251782075776275
MAE:  7.252150678451333
MAE:  7.2429626590774046
MAE:  7.24317

MAE:  6.926635163309633
MAE:  6.929819165643708
MAE:  6.914206859479888
MAE:  6.9211900903862595
MAE:  7.3878878960332335
MAE:  7.463171899497815
MAE:  7.41182757969371
MAE:  7.617190353030714
MAE:  7.533966328344704
MAE:  7.534876324174255
MAE:  7.5355378204589645
MAE:  7.535940636427136
MAE:  7.222505747135893
MAE:  7.274287668606976
MAE:  7.274307258041037
MAE:  7.188019718819257
MAE:  7.199738418318396
MAE:  7.185897815076011
MAE:  7.174302577044213
MAE:  7.174992571249657
MAE:  7.190134949288126
MAE:  6.907442270352742
MAE:  6.730934545765969
MAE:  6.730829889836038
MAE:  6.734636019324975
MAE:  6.83987200001548
MAE:  6.839218283877263
MAE:  6.905455356970303
MAE:  6.874026728173502
MAE:  6.906396390470863
MAE:  6.926600409698858
MAE:  7.088557320601611
MAE:  7.172234952008778
MAE:  7.158079619463379
MAE:  7.160934854710948
MAE:  7.1774269057417275
MAE:  7.176857117522335
MAE:  7.176680833322764
MAE:  7.166281768904587
MAE:  7.22343616760228
MAE:  7.224119768441518
MAE:  7.0193083

MAE:  6.9384132067725375
MAE:  6.961967027235051
MAE:  6.957872965439717
MAE:  6.944568615521744
MAE:  7.304707066925144
MAE:  7.306654693362465
MAE:  7.496677429391282
MAE:  7.4977955974062365
MAE:  7.474228567214082
MAE:  8.019783052059784
MAE:  7.985305329393333
MAE:  7.97381249797938
MAE:  8.021302355225577
MAE:  8.02679134844828
MAE:  8.08875873975486
MAE:  8.086604200329624
MAE:  8.119105619720909
MAE:  7.259007931828775
MAE:  7.25871984762093
MAE:  7.091375819717673
MAE:  7.024842003195455
MAE:  7.354220136379949
MAE:  7.353926323563538
MAE:  7.39358014524211
MAE:  7.356228519300509
MAE:  7.356179841654771
MAE:  7.372729847808443
MAE:  7.305429304324391
MAE:  7.3055955470978455
MAE:  6.802993385829933
MAE:  6.802543029870209
MAE:  6.763761743362131
MAE:  6.789271156530344
MAE:  7.37565510164621
MAE:  7.369519779946131
MAE:  7.393826368101509
MAE:  7.462167556830008
MAE:  7.464113142494681
MAE:  7.453385684947366
MAE:  7.4506891822188726
MAE:  6.761154556880006
MAE:  6.7546230203

...

MAE:  6.9077432315404765
MAE:  7.467201195914576
MAE:  7.768302092178547
MAE:  7.382061346316808
MAE:  7.353019525590767
MAE:  7.656616484782029
MAE:  7.6570295960952075
MAE:  7.654451254756999
MAE:  7.653626913761917
MAE:  7.631824170695137
MAE:  7.633291604402169
MAE:  7.637178517534325
MAE:  7.627011160379021
MAE:  7.408905843859271
MAE:  7.409247109759726
MAE:  7.391201734704156
MAE:  7.411816449173283
MAE:  7.3773371807634
MAE:  7.972896326128141
MAE:  7.561568310376829
MAE:  7.561448174842706
MAE:  7.588764547606554
MAE:  7.589005228741775
MAE:  7.571171230548354
MAE:  7.570705326495772
MAE:  7.569887832386858
MAE:  7.651607830050013
MAE:  7.647871698675392
MAE:  7.978876957735885
MAE:  7.967864827654062
MAE:  8.689285610660852
MAE:  8.700050291207106
MAE:  8.699836846455417
MAE:  8.696506849922548
MAE:  8.678301095213286
MAE:  8.196844399188947
MAE:  7.651849177246565
MAE:  7.47884479551203
MAE:  7.175240746278546
MAE:  6.877715897871233
MAE:  6.900863271796549
MAE:  7.069657986

MAE:  7.892410094618556
MAE:  7.964391597790548
MAE:  7.162282614258759
MAE:  7.205184743495033
MAE:  6.922122770360074
MAE:  6.716238237976577
MAE:  6.716070356201596
MAE:  6.684404160484952
MAE:  6.6855439629455455
MAE:  6.706974208358145
MAE:  6.701000820411154
MAE:  6.700638430127387
MAE:  7.141672995050545
MAE:  6.936737813864341
MAE:  6.935979916579839
MAE:  6.9919113159314366
MAE:  6.992495884576803
MAE:  6.946306101879239
MAE:  6.945820960600546
MAE:  6.942330419036906
MAE:  6.942331072320669
MAE:  6.962646094627094
MAE:  6.624239426212618
MAE:  6.624214306414583
MAE:  6.6227276523231104
MAE:  6.622821365204824
MAE:  6.64429532081838
MAE:  6.783842448742952
MAE:  6.800566893411653
MAE:  6.962764390046047
MAE:  6.977596257676076
MAE:  6.952906061547128
MAE:  6.818910897217085
MAE:  6.766410046237455
MAE:  6.766529539279736
MAE:  6.756876395203607
MAE:  6.782341585575037
MAE:  6.781194661343917
MAE:  6.781325094854513
MAE:  6.777158219862325
MAE:  6.775304141674518
MAE:  7.086078

MAE:  7.532731817536415
MAE:  7.5568460954892975
MAE:  7.5565886636123665
MAE:  7.534922331614756
MAE:  7.534806962002794
MAE:  7.541007461675363
MAE:  6.816078217246785
MAE:  6.783203635280887
MAE:  6.783056463265027
MAE:  6.780770009264461
MAE:  6.575975728006922
MAE:  6.591928572893723
MAE:  6.616374307732528
MAE:  6.681599292276315
MAE:  6.760178579906371
MAE:  6.963934883409667
MAE:  6.964541014722217
MAE:  6.965184614523689
MAE:  6.968712777540131
MAE:  6.8677380246660045
MAE:  6.878527553105542
MAE:  6.858702999994544
MAE:  6.8701128726585
MAE:  6.937866746886387
MAE:  6.957445930229139
MAE:  6.958270816440352
MAE:  6.958262295864113
MAE:  7.105194763291365
MAE:  7.10494233607802
MAE:  7.105150981948946
MAE:  7.104719253046463
MAE:  7.41694160575137
MAE:  7.410364951494103
MAE:  7.410368194991098
MAE:  7.535209959711912
MAE:  7.5509218924944514
MAE:  7.529338270447701
MAE:  7.532338648181426
MAE:  7.728228344315924
MAE:  7.747722817139897
MAE:  8.313939036184028
MAE:  7.46349234

MAE:  7.277136343206843
MAE:  7.301480250756223
MAE:  7.379275395369795
MAE:  7.634341103263247
MAE:  7.639351117834913
MAE:  7.661433431820004
MAE:  7.661415871007334
MAE:  7.695045748292434
MAE:  7.644726563385865
MAE:  7.62526878925502
MAE:  7.2716689765012905
MAE:  7.29379071504596
MAE:  7.3294887304359415
MAE:  7.3452768346240385
MAE:  7.355373525875859
MAE:  7.372960360505941
MAE:  7.291578263514721
MAE:  7.289823562699367
MAE:  7.283677694892191
MAE:  7.258387125119957
MAE:  7.259507008155814
MAE:  7.391584401708391
MAE:  7.4588316965586205
MAE:  7.527989310601968
MAE:  7.50668330750104
MAE:  7.500031728688395
MAE:  7.495096421235547
MAE:  7.480996169485055
MAE:  7.439407552832723
MAE:  7.4577094737798335
MAE:  7.454974612775737
MAE:  7.119613066724268
MAE:  6.914535464071131
MAE:  6.9137208423603305
MAE:  6.901654440648922
MAE:  6.9028189632084
MAE:  6.9015230566303805
MAE:  6.6980956151041555
MAE:  6.701583864576115
MAE:  6.711168138149744
MAE:  6.682545878783167
MAE:  6.68715

MAE:  7.11536185880736
MAE:  6.77872040279695
MAE:  6.784735136829003
MAE:  6.779904207243531
MAE:  6.749954623904984
MAE:  6.752534542355802
MAE:  6.59383992138789
MAE:  6.610818863911887
MAE:  6.607105372580707
MAE:  6.827639409162024
MAE:  6.759470342835498
MAE:  6.765858236688798
MAE:  6.775056201721123
MAE:  6.775368991742237
MAE:  6.77769788597013
MAE:  7.3424139437421365
MAE:  7.342525006854532
MAE:  7.343262269661699
MAE:  7.371022914289502
MAE:  7.360377714034171
MAE:  7.356708331852257
MAE:  7.356135713140706
MAE:  7.3554782546818975
MAE:  7.356743835079797
MAE:  7.357008390026547
MAE:  7.329065197920707
MAE:  7.163026468232684
MAE:  7.191943465561857
MAE:  7.1920405902820885
MAE:  7.594642263058449
MAE:  7.018626795159275
MAE:  6.947068508374617
MAE:  6.826777181366088
MAE:  6.740853589265348
MAE:  6.775072554070017
MAE:  6.769029908397438
MAE:  7.662333253460572
MAE:  7.6371170168282685
MAE:  7.63833067018754
MAE:  7.6990893427772855
MAE:  7.699326299694864
MAE:  7.68303768

In [1]:
b = [[-8.08714909, -4.08714909] ,
                [-8.14730804, -4.14730804] ,
                [-7.98892457, -3.98892457] ,
                [-10.96016772, -6.960167719999999] ,
                [-6.47571419, -2.4757141899999997] ,
                [-10.61162735, -6.611627349999999] ,
                [-8.51695205, -4.51695205] ,
                [-5.52932059, -1.5293205900000002] ,
                [-5.87109302, -1.87109302] ,
                [-7.93522543, -3.93522543] ,
                [-8.538020450000001, -4.53802045] ,
                [-8.25750151, -4.25750151] ,
                [-5.12460613, -1.12460613] ,
                [-8.49405602, -4.49405602] ,
                [-8.73598517, -4.73598517] ,
                [-11.13666712, -7.13666712] ,
                [-5.66514436, -1.6651443600000002] ,
                [-7.7455947, -3.7455947] ,
                [-7.35777882, -3.35777882] ,
                [-9.98832238, -5.98832238] ]

for i in b:
    print('[',10**i[0],'-',10**i[1],']')

[ 8.181838636063187e-09 - 8.181838636063204e-05 ]
[ 7.123475910762391e-09 - 7.123475910762392e-05 ]
[ 1.0258300810448363e-08 - 0.00010258300810448363 ]
[ 1.0960548294532955e-11 - 1.0960548294532954e-07 ]
[ 3.3441504678417514e-07 - 0.003344150467841751 ]
[ 2.4455280584079124e-11 - 2.4455280584079123e-07 ]
[ 3.041220785149957e-09 - 3.041220785149957e-05 ]
[ 2.955829709624887e-06 - 0.029558297096248867 ]
[ 1.3455721198528924e-06 - 0.013455721198528925 ]
[ 1.1608458951077635e-08 - 0.00011608458951077636 ]
[ 2.897207161189201e-09 - 2.897207161189207e-05 ]
[ 5.527114862165974e-09 - 5.5271148621659846e-05 ]
[ 7.505746112699534e-06 - 0.07505746112699535 ]
[ 3.205855771878882e-09 - 3.205855771878882e-05 ]
[ 1.8366010573126943e-09 - 1.8366010573126904e-05 ]
[ 7.300168424807389e-12 - 7.30016842480739e-08 ]
[ 2.1619997529957594e-06 - 0.021619997529957597 ]
[ 1.7964093217723464e-08 - 0.00017964093217723463 ]
[ 4.3875409210039854e-08 - 0.0004387540921003985 ]
[ 1.0272534771981023e-10 - 1.02725347719

In [ ]:
b[0]

In [11]:
from tqdm import tqdm
model_outputs_meters = model_outputs/9810

# Create an empty array to store the results
mu_star = np.zeros([problem['num_vars'],np.shape(model_outputs)[1]])
sigma = np.zeros([problem['num_vars'],np.shape(model_outputs)[1]])

# Iterate over each output column
for i in tqdm(range(np.shape(model_outputs_meters)[1]), desc="Progress", unit="output"):
    # Perform Morris analysis for each output column
    morris_indices = morris.analyze(problem, param_values, model_outputs_meters[:,i],
                                    conf_level=0.95, print_to_console=False)
    
    # Store the results in the respective arrays
    mu_star[:,i] = morris_indices['mu_star']
    sigma[:,i] = morris_indices['sigma']

meshP = pv.read(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po_ts_1_t_1.000000.vtu")
meshMu2 = meshP
for par in range(problem['num_vars']):
    colNameMu = f'mu_star_P{par+1}'
    colNameS = f'sigma_P{par+1}'
    meshMu2[colNameMu] = mu_star[par,:]
    meshMu2[colNameS] = sigma[par,:]
    
output_path = r'D:\Andrea\Flow\SensitivityAnalisys\meshSA2.vtu'
meshMu2.save(output_path)



Progress: 100%|██████████████████████████████████████████████████████████████| 36204/36204 [22:19<00:00, 27.03output/s]


In [ ]:
import numpy as np
import SALib

# Define your model evaluation function
evaluate_model = model1

# Define the problem definition
problem = {
    'num_vars': 7,  # Number of input parameters
    'names': ['param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'param7'],  # Parameter names
    'bounds': [[-9.203212302198214,-8.203212302198214],
                [-8.557603941922235,-7.557603941922235],
                [-12.960415758067137,-11.960415758067137],
                [-9.880580941153951,-8.880580941153951],
                [-10.681610259869824,-9.681610259869824],
                [-17.480981875487103,-16.480981875487103],
                [ 1.691223510483059,2.291223510483059]],  # Parameter bounds
    'groups': None  # Parameter grouping (optional)
}

# Define the number of samples and levels for the Morris method
n_samples = 1000
n_levels = 4

# Generate the Morris samples using the provided sampler

param_values = SALib.sample.morris.sample(problem, n_samples, n_levels, optimal_trajectories=True, local_optimization=True, seed=10) 

# Evaluate the model for each sample
model_outputs = np.array([evaluate_model(x,args1) for x in param_values])

# Perform the Morris analysis
morris_indices = morris.analyze(problem, param_values, model_outputs,
                                conf_level=0.95, print_to_console=True)

# Print the Morris indexes
print("Morris Indexes:")
for param, indexes in zip(problem['names'], morris_indices['mu_star']):
    print(f"{param}: {indexes}")


In [74]:
import numpy as np
import math

# Define your model evaluation function
def evaluate_model_test(x):
    y = [1*math.sin(x[0]),2*math.sin(x[1]),3*math.sin(x[2]),4*math.sin(x[3]),5*math.sin(x[4]),6*math.sin(x[5]),7*math.sin(x[6])]
    return y

# Define the problem definition
problem_test = {
    'num_vars': 7,  # Number of input parameters
    'names': ['param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'param7'],  # Parameter names
    'bounds': [[0, 1]] * 7,  # Parameter bounds
    'groups': None  # Parameter grouping (optional)
}

# Define the number of samples and levels for the Campolongo method
n_samples = 1000
n_levels = 4

# Generate the Morris samples using the provided sampler
param_values_test = morris_sampler.sample(problem_test, n_samples, n_levels)

# Evaluate the model for each sample
model_outputs_test = np.array([evaluate_model_test(x) for x in param_values_test])

# Perform the Morris analysis
morris_indices = morris.analyze(problem_test, param_values, model_outputs_test,
                                conf_level=0.95, print_to_console=True)

# Print the Campolongo indexes
print("Campolongo Indexes:")
for param, indexes in zip(problem['names'], campolongo_indices['mu_star']):
    print(f"{param}: {indexes}")


ValueError: cannot reshape array of size 56000 into shape (1000,8)

array([0.33333333, 0.66666667, 0.66666667, 1.        , 0.33333333,
       0.33333333, 0.33333333])